# PREDICTION USING LOGISTIC REGRESSION, RANDOM FOREST, GRADIENT BOOSTED TREES, SUPPORT VECTOR MACHINE, ARTIFICIAL NEURAL NETWORK

In [ ]:
# library requirements
# pip install sklearn
# pip install imbalanced-learn
# pip install delayed
# pip install graphviz
# pip install keras
# pip install tensorflow --user

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from numpy import isnan
from sklearn import metrics
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import *
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from IPython.display import display
from plotnine import *
import keras
from keras import models
from keras import layers
from keras.models import Sequential
from keras.constraints import maxnorm
from keras.layers import Dense, Dropout, Activation
from numpy.random import seed
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn import tree
from sklearn.tree import export_graphviz, DecisionTreeClassifier
import graphviz # need to install graphviz on system https://www.graphviz.org/download/ 
import os

# open dataframe
df = pd.read_csv("/PATH/TO/FILE.csv")
# define imputer
imputer = IterativeImputer(estimator=BayesianRidge(), # default
                           n_nearest_features=None, # default
                           imputation_order='ascending', # uses features with fewest missing values first
                           initial_strategy='most_frequent') # used for categorical data
# fit the imputer on the dataset
imputer.fit(df)
# transform the dataset
transformed_df = imputer.transform(df)
# print total missing
print('Missing: %d' % sum(isnan(transformed_df).flatten()))
# create complete dataframe with imputed data
complete_df = pd.DataFrame(imputer.transform(df))
complete_df.columns = [#insert column names#]
completed_df = complete_df.round()
# define the modeling pipeline
model = RandomForestClassifier()
imputer = IterativeImputer()
pipeline = Pipeline(steps=[('i', imputer), ('m', model)])
# split into input and output elements
data = df.values
ix = [i for i in range(data.shape[1])]
X, y = data[:, ix], data[:, 1] # 1 is based on 'Variable' column
# define model evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Mean Accuracy: %.3f (%.3f)'% (scores.mean(), scores.std()))
# test imputation on one variable
grade_unrounded = complete_df['Variable'].value_counts() #insert veriable name
# print("Frequency of Variable")
# print(grade_unrounded)
grade_rounded = completed_df['Variable'].value_counts()
# print("Frequency of Variable")
# print(grade_rounded)

# split the dataset into input and output features
X = completed_df.drop(axis=1, columns=['Variable']) #drop outcome variable and other variables not to be used in prediction
y = completed_df['Variable'] #enter outcome variable
# create balanced dataset
smote = SMOTE(random_state=1, k_neighbors=5)
X_smote, y_smote = smote.fit_resample(X, y)
# check dataset is balanced
# print(f'''Shape of X before SMOTE: {X.shape}
# Shape of X after SMOTE: {X_smote.shape}''')
# print(f'''Shape of y before SMOTE: {y.shape}
# Shape of y after SMOTE: {y_smote.shape}''')
# print('\nBalance of positive and negative classes (%):')
y_smote.value_counts(normalize=True) * 100

# split data into train and test set, default 20% test, set random_state = 0 for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, stratify=y_smote, random_state=0, test_size = 0.2)

# create logistic regression model
Log_Reg = LogisticRegression(random_state=0)
# use gridsearchCV to find the best parameters for the linear model, define values for parameter tuning, change as needed 
penalty = ['l1', 'l2']
C = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.25, 0.5, 0.75, 1.0]
class_weight = [{0:.1, 1:.9}, {0:.2, 1:.8}, {0:.3, 1:.7}, {0:.4, 1:.6}, {0:.5, 1:.5}]
max_iter = [50, 100, 200, 500]
# create Log Reg parameter dictionary for tuning
Log_Reg_param_grid = dict(penalty=penalty,
                          C=C, 
                          class_weight=class_weight,
                          max_iter=max_iter)
# define the Log Reg gridsearchCV object with 5 CV
grid_search_Log_Reg = GridSearchCV(Log_Reg, param_grid=Log_Reg_param_grid, n_jobs=-1, cv=5, scoring='accuracy')
# fit the Log Reg gridsearchCV object
grid_search_Log_Reg.fit(X_train, y_train)
# print best Log Reg parameters and average accuracy score
print("Best parameters: {}".format(grid_search_Log_Reg.best_params_))
print("Best cross-validation average accuracy score: {:.2f}".format(grid_search_Log_Reg.best_score_))
# predict outcome using tuned log reg model on train set
y_pred_Log_Reg_grid_search_train= grid_search_Log_Reg.predict(X_train)
# predict outcome using tuned log reg model on train set
y_pred_Log_Reg_grid_search = grid_search_Log_Reg.predict(X_test)
# print the accuracy scores
print("Accuracy on training set: {:.2f}".format(accuracy_score(y_train, y_pred_Log_Reg_grid_search_train)))
print("Accuracy on test set: {:.2f}".format(accuracy_score(y_test, y_pred_Log_Reg_grid_search)))
# evaluate performance metrics of the tuned linear model
# construct the confusion matrix
cm_Log_Reg = confusion_matrix(y_true = y_test, y_pred = y_pred_Log_Reg_grid_search)
# plot confusion matrix
print("Confusion matrix plot of the LogReg:")
print(cm_Log_Reg)
print()
print("Confusion matrix plot of the LogReg:")
disease_labels = ['Outcome Absent', 'Outcome Present']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm_Log_Reg)
plt.title('Confusion matrix of the LogReg:')
fig.colorbar(cax)
ax.set_xticklabels([''] + disease_labels)
ax.set_yticklabels([''] + disease_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()
# print classification report
print("Precision, Recall, F1-score for class 0 (Outcome Absent) and class 1 (Outcome Present) classes of the test set:")
print("Using logistic regression (l2) model")
print()
print(classification_report(y_test, y_pred_Log_Reg_grid_search))
# plot area under curve for LogReg model
y_pred_proba_Log_Reg = grid_search_Log_Reg.predict_proba(X_test)[:,1]
print("Area under curve for Log Reg:")
print(metrics.roc_auc_score(y_true = y_test, y_score = y_pred_proba_Log_Reg))
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_proba_Log_Reg)
plt.figure()
plt.plot(fpr, tpr, color='red', lw=2, label='ROC curve LogReg')
plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()

# fit a gradient boosted tree model
gbt = GradientBoostingClassifier(random_state=0)
# use gridsearchCV to find the best parameters for the gradient boosted tree model, define values for parameter tuning, change as needed  
learning_rate = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.5, 1]
n_estimators = [100, 500, 1000, 2000, 5000]
max_depth = [2, 4, 6, 8, 10]
min_samples_split = [2, 5, 10]
# create gradient boosted tree parameter dictionary for tuning
gbt_param_grid = dict(learning_rate=learning_rate,
                            n_estimators=n_estimators, 
                            max_depth=max_depth,
                            min_samples_split=min_samples_split)
# define the gradient boosted tree gridsearchCV object with 5 CV
grid_search_gbt = GridSearchCV(gbt, param_grid=gbt_param_grid, cv=5, n_jobs=-1, scoring = 'accuracy')
# fit the tree gridsearchCV object
grid_search_gbt.fit(X_train, y_train)
# print best gradient boosted tree parameters and average accuracy score
print("Best parameters: {}".format(grid_search_gbt.best_params_))
print("Best cross-validation average accuracy score: {:.2f}".format(grid_search_gbt.best_score_))
# predict outcome using tuned gradient boosted model on train set
y_pred_gbt_grid_search_train= grid_search_gbt.predict(X_train)
# predict outcome using tuned gradient boost model on train set
y_pred_gbt_grid_search = grid_search_gbt.predict(X_test)
# print the accuracy scores
print("Accuracy on training set: {:.2f}".format(accuracy_score(y_train, y_pred_gbt_grid_search_train)))
print("Accuracy on test set: {:.2f}".format(accuracy_score(y_test, y_pred_gbt_grid_search)))
# evaluate performance metrics of the tuned gbt model
# construct the confusion matrix
cm_gbt = confusion_matrix(y_true = y_test, y_pred = y_pred_gbt_grid_search)
# plot confusion matrix
print("Confusion matrix plot of the gbt:")
print(cm_gbt)
print()
print("Confusion matrix plot of the gbt:")
disease_labels = ['Outcome Absent', 'Outcome Present']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm_gbt)
plt.title('Confusion matrix of the gbt:')
fig.colorbar(cax)
ax.set_xticklabels([''] + disease_labels)
ax.set_yticklabels([''] + disease_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()
# print classification report
print("Precision, Recall, F1-score for class 0 (Outcome Absent) and class 1 (Outcome Present) classes of the test set:")
print("Using gradient boosted tree model")
print()
print(classification_report(y_test, y_pred_gbt_grid_search))
# plot area under curve for gbt model
y_pred_proba_gbt = grid_search_gbt.predict_proba(X_test)[:,1]
print("Area under curve for gradient boosted tree:")
print(metrics.roc_auc_score(y_true = y_test, y_score = y_pred_proba_gbt))
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_proba_gbt)
plt.figure()
plt.plot(fpr, tpr, color='red', lw=2, label='ROC curve gradient boosted tree')
plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()

# fit a random forrest tree model
RF = RandomForestClassifier(random_state=0) 
# use gridsearchCV to find the best parameters for the random forest tree model, define values for parameter tuning, change as needed  
max_features = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
n_estimators = [1000, 2000, 5000, 10000]
max_depth = [2, 4, 6, 8, 10]
class_weight = [{0:.1, 1:.9}, {0:.2, 1:.8}, {0:.3, 1:.7}, {0:.4, 1:.6}, {0:.5, 1:.5}]
# create random forest tree parameter dictionary for tuning
RF_param_grid = dict(max_features=max_features,
                            n_estimators=n_estimators, 
                            max_depth=max_depth,
                            class_weight=class_weight)
# define the random forest tree gridsearchCV object with 5 CV
grid_search_RF = GridSearchCV(RF, param_grid=RF_param_grid, cv=5, n_jobs=-1, scoring = 'accuracy')
# fit the tree gridsearchCV object
grid_search_RF.fit(X_train, y_train)
# print best random forest tree parameters and average accuracy score
print("Best parameters: {}".format(grid_search_RF.best_params_))
print("Best cross-validation average accuracy score: {:.2f}".format(grid_search_RF.best_score_))
# predict outcome using tuned random forest model on train set
y_pred_RF_grid_search_train= grid_search_RF.predict(X_train)
# predict outcome using tuned RF model on train set
y_pred_RF_grid_search = grid_search_RF.predict(X_test)
# print the accuracy scores
print("Accuracy on training set: {:.2f}".format(accuracy_score(y_train, y_pred_RF_grid_search_train)))
print("Accuracy on test set: {:.2f}".format(accuracy_score(y_test, y_pred_RF_grid_search)))
# evaluate performance metrics of the tuned random forest model
# construct the confusion matrix
cm_RF = confusion_matrix(y_true = y_test, y_pred = y_pred_RF_grid_search)
# plot confusion matrix
print("Confusion matrix plot of the RF:")
print(cm_RF)
print()
print("Confusion matrix plot of the RF:")
disease_labels = ['Outcome Absent', 'Outcome Present']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm_RF)
plt.title('Confusion matrix of the RF:')
fig.colorbar(cax)
ax.set_xticklabels([''] + disease_labels)
ax.set_yticklabels([''] + disease_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()
# print classification report
print("Precision, Recall, F1-score for class 0 (Outcome Absent) and class 1 (Outcome Present) classes of the test set:")
print("Using random forest tree model")
print()
print(classification_report(y_test, y_pred_RF_grid_search))
# plot area under curve for RF model
y_pred_proba_RF = grid_search_RF.predict_proba(X_test)[:,1]
print("Area under curve for random forest tree:")
print(metrics.roc_auc_score(y_true = y_test, y_score = y_pred_proba_RF))
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_proba_RF)
plt.figure()
plt.plot(fpr, tpr, color='red', lw=2, label='ROC curve random forest tree')
plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()

# fit a SVM model with scaled data 
svm = SVC(probability=True, random_state=0)
# use gridsearchCV to find the best parameters for the svm model, define values for parameter tuning, change as needed  
C = [1/100000, 1/10000, 1/1000, 1/100, 1/10]
kernel = ['linear', 'sigmoid', 'poly', 'rbf'] 
gamma = [1, 1/2, 1/3, 1/5, 1/10, 1/50, 1/100]
class_weight = [{0:.1, 1:.9}, {0:.2, 1:.8}, {0:.3, 1:.7}, {0:.4, 1:.6}, {0:.5, 1:.5}]
# create random forest tree parameter dictionary for tuning
svm_param_grid = dict(C=C,
                        kernel=kernel, 
                        gamma=gamma,
                        class_weight=class_weight)
# define the svm gridsearchCV object with 5 CV with accuracy
grid_search_svm = GridSearchCV(svm, param_grid=svm_param_grid, cv=5, n_jobs=-1, scoring='accuracy')
# fit the svm gridsearchCV object
grid_search_svm.fit(X_train, y_train)
# print best svm parameters and average accuracy score
print("Best parameters: {}".format(grid_search_svm.best_params_))
print("Best cross-validation average accuracy score: {:.2f}".format(grid_search_svm.best_score_))
# predict outcome using tuned svm model on train set
y_pred_svm_grid_search_train= grid_search_svm.predict(X_train)
# predict outcome using tuned svm model on train set
y_pred_svm_grid_search = grid_search_svm.predict(X_test)
# print the accuracy scores
print("Accuracy on training set: {:.2f}".format(accuracy_score(y_train, y_pred_svm_grid_search_train)))
print("Accuracy on test set: {:.2f}".format(accuracy_score(y_test, y_pred_svm_grid_search)))
# evaluate performance metrics of the tuned svm model
# construct the confusion matrix
cm_svm = confusion_matrix(y_true = y_test, y_pred = y_pred_svm_grid_search)
# plot confusion matrix
print("Confusion matrix plot of the svm:")
print(cm_svm)
print()
print("Confusion matrix plot of the svm:")
disease_labels = ['Outcome Absent', 'Outcome Present']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm_svm)
plt.title('Confusion matrix of the svm:')
fig.colorbar(cax)
ax.set_xticklabels([''] + disease_labels)
ax.set_yticklabels([''] + disease_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()
# Print classification report
print("Precision, Recall, F1-score for class 0 (Outcome Absent) and class 1 (Outcome Present) classes of the test set:")
print("Using svm model")
print()
print(classification_report(y_test, y_pred_svm_grid_search))
# plot area under curve for svm model
y_pred_proba_svm = grid_search_svm.predict_proba(X_test)[:,1]
print("Area under curve for svm:")
print(metrics.roc_auc_score(y_true = y_test, y_score = y_pred_proba_svm))
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_proba_svm)
plt.figure()
plt.plot(fpr, tpr, color='red', lw=2, label='ROC curve svm')
plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()

# fix random seed to enable experiment to be reproducible, randomly selected 2
seed = 2
tf.random.set_seed(2)
# create and compile ANN model with randomly selected parameters, parameters will be tuned later
def create_model(dropout_rate=0.1, 
                 weight_constraint=1, 
                 optimizer='sgd',
                 learn_rate=0.01,
                 momentum=0.1,
                 neurons=10,
                 activation='relu'):
    model = Sequential()
    # input_dim = number of imput variables
    model.add(Dense(neurons, input_dim=23, kernel_initializer='uniform', activation='linear', kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
# compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
# wrap Keras model so it can be used by scikit-learn 
ann = KerasClassifier(build_fn=create_model, verbose=0)
# define values for each hyperparameter for tuning, change as needed 
dropout_rate = [0.005, 0.01, 0.05]
epochs = [5, 10, 15, 20]
batch_size = [50, 100, 500, 1000]
optimizer = ['SGD'] # select SGD as default for time cost reduction
learn_rate = [0.001, 0.01, 0.1]
momentum = [0.0001, 0.001, 0.01, 0.1]
neurons = [5, 10, 15, 20]
activation = ['sigmoid'] # select sigmoid for binary output

ann_param_grid = dict(dropout_rate=dropout_rate, 
                  batch_size=batch_size, 
                  epochs=epochs,
                  optimizer=optimizer, 
                  learn_rate=learn_rate, 
                  momentum=momentum, 
                  neurons=neurons, 
                  activation=activation)
# define the svm gridsearchCV object with 5 CV with accuracy score
grid_search_ann = GridSearchCV(ann, param_grid=ann_param_grid, cv=5, n_jobs=-1, scoring='accuracy')
# fit grid search to training set
grid_search_ann.fit(X_train, y_train)
# print best ann parameters and average accuracy score
print("Best parameters: {}".format(grid_search_ann.best_params_))
print("Best cross-validation average accuracy score: {:.2f}".format(grid_search_ann.best_score_))
# predict outcome using tuned ann model on train set
y_pred_ann_grid_search_train= grid_search_ann.predict(X_train)
# predict outcome using tuned ann model on train set
y_pred_ann_grid_search = grid_search_ann.predict(X_test)
# print accuracy scores
print("Accuracy on training set: {:.2f}".format(accuracy_score(y_train, y_pred_ann_grid_search_train)))
print("Accuracy on test set: {:.2f}".format(accuracy_score(y_test, y_pred_ann_grid_search)))
# evaluate performance metrics of the tuned ann model
# construct the confusion matrix
cm_ann = confusion_matrix(y_true = y_test, y_pred = y_pred_ann_grid_search)
# plot confusion matrix
print("Confusion matrix plot of the ann:")
print(cm_ann)
print()
print("Confusion matrix plot of the ann:")
disease_labels = ['Outcome Absent', 'Outcome Present']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm_ann)
plt.title('Confusion matrix of the ann:')
fig.colorbar(cax)
ax.set_xticklabels([''] + disease_labels)
ax.set_yticklabels([''] + disease_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()
# print classification report
print("Precision, Recall, F1-score for class 0 (Outcome Absent) and class 1 (Outcome Present) classes of the test set:")
print("Using ann model")
print()
print(classification_report(y_test, y_pred_ann_grid_search))
# plot area under curve for ann model
y_pred_proba_ann = grid_search_ann.predict_proba(X_test)[:,1]
print("Area under curve for ann:")
print(metrics.roc_auc_score(y_true = y_test, y_score = y_pred_proba_ann))
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_proba_ann)
plt.figure()
plt.plot(fpr, tpr, color='red', lw=2, label='ROC curve ann')
plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()

# visualse decision tree
decision_tree = grid_search_RF
print("Accuracy on training set: {:.3f}".format(decision_tree.score(X_train, y_train))) 
print("Accuracy on test set: {:.3f}".format(decision_tree.score(X_test, y_test)))
export_graphviz(decision_tree, out_file="tree.dot", class_names=["Outcome Present", "Outcome Absent"], feature_names=X.columns, impurity=False, filled=True)
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin'#'C:/Users/............../graphviz-2.38/release/bin/'
with open("tree.dot") as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)
# plot feature importances
print("Feature importances:\n{}".format(decision_tree.feature_importances_))
df_feature_importance = pd.DataFrame(decision_tree.feature_importances_, index=X_smote.columns, columns=['feature importance']).sort_values('feature importance', ascending=True)
df_feature_importance
ax = df_feature_importance.plot.barh()